In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import matplotlib.pyplot as plt
import numpy as np

# Define transformations for the training and testing data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load the training and test datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Load the ResNet18 model
model = resnet18(pretrained=False, num_classes=10)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the model
for epoch in range(10):  # Number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[Epoch: {epoch + 1}, Mini-batch: {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Define a hook function to capture the output of each layer
activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# Register hooks for each layer in the ResNet18 model
for name, layer in model.named_children():
    if isinstance(layer, nn.Sequential):
        for sub_name, sub_layer in layer.named_children():
            sub_layer.register_forward_hook(get_activation(f'{name}_{sub_name}'))
    else:
        layer.register_forward_hook(get_activation(name))

# Get a random image from the test set
dataiter = iter(testloader)
images, labels = dataiter.next()
image = images[0].unsqueeze(0).to(device)

# Pass the image through the model
output = model(image)

# Plot the outputs of each layer
for layer_name, layer_output in activation.items():
    print(f'{layer_name}: {layer_output.shape}')

    # Convert the output tensor to a NumPy array
    output_np = layer_output.cpu().numpy()

    # If the output is 4D, we can visualize the first channel's activation map
    if len(output_np.shape) == 4:
        plt.figure(figsize=(15, 15))
        for i in range(min(16, output_np.shape[1])):  # Display up to 16 feature maps
            plt.subplot(4, 4, i + 1)
            plt.imshow(output_np[0, i, :, :], cmap='viridis')
            plt.axis('off')
        plt.suptitle(f'{layer_name} Feature Maps')
        plt.show()


Files already downloaded and verified
Files already downloaded and verified
[Epoch: 1, Mini-batch: 100] loss: 1.991
[Epoch: 1, Mini-batch: 200] loss: 1.674
[Epoch: 1, Mini-batch: 300] loss: 1.553
[Epoch: 2, Mini-batch: 100] loss: 1.336
[Epoch: 2, Mini-batch: 200] loss: 1.314
[Epoch: 2, Mini-batch: 300] loss: 1.291
[Epoch: 3, Mini-batch: 100] loss: 1.135
[Epoch: 3, Mini-batch: 200] loss: 1.102
[Epoch: 3, Mini-batch: 300] loss: 1.113
[Epoch: 4, Mini-batch: 100] loss: 0.944
[Epoch: 4, Mini-batch: 200] loss: 0.945
[Epoch: 4, Mini-batch: 300] loss: 0.957
[Epoch: 5, Mini-batch: 100] loss: 0.780
[Epoch: 5, Mini-batch: 200] loss: 0.804
[Epoch: 5, Mini-batch: 300] loss: 0.809
[Epoch: 6, Mini-batch: 100] loss: 0.631
[Epoch: 6, Mini-batch: 200] loss: 0.662
[Epoch: 6, Mini-batch: 300] loss: 0.664
[Epoch: 7, Mini-batch: 100] loss: 0.482
[Epoch: 7, Mini-batch: 200] loss: 0.509
[Epoch: 7, Mini-batch: 300] loss: 0.554
[Epoch: 8, Mini-batch: 100] loss: 0.365
[Epoch: 8, Mini-batch: 200] loss: 0.372
[Epo

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512, 1, 1])